In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15646, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 15646 (delta 127), reused 131 (delta 62), pack-reused 15393
Receiving objects: 100% (15646/15646), 14.65 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (10655/10655), done.


In [ ]:
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 954.1 kB/s eta 0:00:00


In [ ]:
import torch
from IPython.display import Image  
import os 
import random
import shutil

import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.1/562.1 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 27.6 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/MahmudulAlam/Complete-Blood-Cell-Count-Dataset.git

Cloning into 'Complete-Blood-Cell-Count-Dataset'...
remote: Enumerating objects: 841, done.
remote: Total 841 (delta 0), reused 0 (delta 0), pack-reused 841
Receiving objects: 100% (841/841), 7.21 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (402/402), done.


In [ ]:
# to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    info_dict = {}
    info_dict['bboxes'] = []
    for elem in root:
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)

        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    print(info_dict)     
    return info_dict

In [ ]:
# Maping names with IDs
class_name_to_id_mapping = {"RBC": 0,
                           "WBC": 1,
                           "Platelets": 2
                           }


def convert_to_yolov5(info_dict,file):
    print_buffer = []
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    save_file_name = os.path.join(f"/content/Complete-Blood-Cell-Count-Dataset/{file}/Annotations", info_dict["filename"].replace("jpg", "txt"))
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# Get the annotations
for i in ['Training','Validation','Testing']:
  annotations = [os.path.join(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations", x) for x in os.listdir(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations") if x[-3:] == "xml"]
  annotations.sort()
  for ann in tqdm(annotations):
      info_dict = extract_info_from_xml(ann)
      convert_to_yolov5(info_dict,i)
  annotations = [os.path.join(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations", x) for x in os.listdir(f"/content/Complete-Blood-Cell-Count-Dataset/{i}/Annotations") if x[-3:] == "txt"]

 71%|███████   | 213/300 [00:00<00:00, 1086.85it/s]

{'bboxes': [{'class': 'WBC', 'xmin': 260, 'ymin': 177, 'xmax': 491, 'ymax': 376}, {'class': 'RBC', 'xmin': 78, 'ymin': 336, 'xmax': 184, 'ymax': 435}, {'class': 'RBC', 'xmin': 63, 'ymin': 237, 'xmax': 169, 'ymax': 336}, {'class': 'RBC', 'xmin': 214, 'ymin': 362, 'xmax': 320, 'ymax': 461}, {'class': 'RBC', 'xmin': 414, 'ymin': 352, 'xmax': 506, 'ymax': 445}, {'class': 'RBC', 'xmin': 555, 'ymin': 356, 'xmax': 640, 'ymax': 455}, {'class': 'RBC', 'xmin': 469, 'ymin': 412, 'xmax': 567, 'ymax': 480}, {'class': 'RBC', 'xmin': 1, 'ymin': 333, 'xmax': 87, 'ymax': 437}, {'class': 'RBC', 'xmin': 4, 'ymin': 406, 'xmax': 95, 'ymax': 480}, {'class': 'RBC', 'xmin': 155, 'ymin': 74, 'xmax': 247, 'ymax': 174}, {'class': 'RBC', 'xmin': 11, 'ymin': 84, 'xmax': 104, 'ymax': 162}, {'class': 'RBC', 'xmin': 534, 'ymin': 39, 'xmax': 639, 'ymax': 139}, {'class': 'RBC', 'xmin': 547, 'ymin': 195, 'xmax': 640, 'ymax': 295}, {'class': 'RBC', 'xmin': 388, 'ymin': 11, 'xmax': 481, 'ymax': 111}, {'class': 'RBC', 'xmi

100%|██████████| 300/300 [00:00<00:00, 1047.11it/s]


{'bboxes': [{'class': 'RBC', 'xmin': 316, 'ymin': 202, 'xmax': 426, 'ymax': 276}, {'class': 'RBC', 'xmin': 370, 'ymin': 17, 'xmax': 460, 'ymax': 123}, {'class': 'RBC', 'xmin': 176, 'ymin': 2, 'xmax': 291, 'ymax': 113}, {'class': 'RBC', 'xmin': 1, 'ymin': 232, 'xmax': 97, 'ymax': 364}, {'class': 'RBC', 'xmin': 88, 'ymin': 217, 'xmax': 192, 'ymax': 324}, {'class': 'RBC', 'xmin': 202, 'ymin': 194, 'xmax': 304, 'ymax': 283}, {'class': 'RBC', 'xmin': 380, 'ymin': 273, 'xmax': 476, 'ymax': 389}, {'class': 'RBC', 'xmin': 470, 'ymin': 214, 'xmax': 560, 'ymax': 363}, {'class': 'RBC', 'xmin': 559, 'ymin': 212, 'xmax': 640, 'ymax': 358}, {'class': 'RBC', 'xmin': 513, 'ymin': 75, 'xmax': 625, 'ymax': 177}, {'class': 'RBC', 'xmin': 354, 'ymin': 116, 'xmax': 467, 'ymax': 204}, {'class': 'RBC', 'xmin': 396, 'ymin': 392, 'xmax': 509, 'ymax': 480}, {'class': 'Platelets', 'xmin': 127, 'ymin': 156, 'xmax': 167, 'ymax': 201}, {'class': 'WBC', 'xmin': 132, 'ymin': 274, 'xmax': 355, 'ymax': 480}], 'filename

100%|██████████| 60/60 [00:00<00:00, 1118.90it/s]


{'bboxes': [{'class': 'RBC', 'xmin': 385, 'ymin': 98, 'xmax': 523, 'ymax': 198}, {'class': 'RBC', 'xmin': 384, 'ymin': 164, 'xmax': 499, 'ymax': 260}, {'class': 'RBC', 'xmin': 101, 'ymin': 120, 'xmax': 224, 'ymax': 222}, {'class': 'RBC', 'xmin': 130, 'ymin': 344, 'xmax': 234, 'ymax': 443}, {'class': 'RBC', 'xmin': 161, 'ymin': 381, 'xmax': 254, 'ymax': 480}, {'class': 'RBC', 'xmin': 14, 'ymin': 228, 'xmax': 123, 'ymax': 344}, {'class': 'RBC', 'xmin': 306, 'ymin': 293, 'xmax': 415, 'ymax': 409}, {'class': 'RBC', 'xmin': 531, 'ymin': 103, 'xmax': 632, 'ymax': 221}, {'class': 'RBC', 'xmin': 492, 'ymin': 233, 'xmax': 593, 'ymax': 347}, {'class': 'RBC', 'xmin': 364, 'ymin': 261, 'xmax': 465, 'ymax': 375}, {'class': 'RBC', 'xmin': 264, 'ymin': 60, 'xmax': 365, 'ymax': 174}, {'class': 'RBC', 'xmin': 249, 'ymin': 174, 'xmax': 369, 'ymax': 296}, {'class': 'WBC', 'xmin': 283, 'ymin': 1, 'xmax': 567, 'ymax': 106}, {'class': 'RBC', 'xmin': 109, 'ymin': 1, 'xmax': 202, 'ymax': 98}, {'class': 'RBC',

  0%|          | 0/60 [00:00<?, ?it/s]

{'bboxes': [{'class': 'RBC', 'xmin': 378, 'ymin': 278, 'xmax': 489, 'ymax': 374}, {'class': 'RBC', 'xmin': 448, 'ymin': 67, 'xmax': 580, 'ymax': 177}, {'class': 'RBC', 'xmin': 308, 'ymin': 23, 'xmax': 409, 'ymax': 145}, {'class': 'RBC', 'xmin': 10, 'ymin': 220, 'xmax': 117, 'ymax': 337}, {'class': 'RBC', 'xmin': 65, 'ymin': 190, 'xmax': 181, 'ymax': 319}, {'class': 'RBC', 'xmin': 171, 'ymin': 66, 'xmax': 279, 'ymax': 185}, {'class': 'RBC', 'xmin': 136, 'ymin': 329, 'xmax': 259, 'ymax': 442}, {'class': 'RBC', 'xmin': 229, 'ymin': 322, 'xmax': 350, 'ymax': 414}, {'class': 'RBC', 'xmin': 25, 'ymin': 368, 'xmax': 160, 'ymax': 477}, {'class': 'RBC', 'xmin': 370, 'ymin': 382, 'xmax': 489, 'ymax': 477}, {'class': 'RBC', 'xmin': 525, 'ymin': 8, 'xmax': 638, 'ymax': 93}, {'class': 'RBC', 'xmin': 20, 'ymin': 28, 'xmax': 141, 'ymax': 147}, {'class': 'RBC', 'xmin': 510, 'ymin': 259, 'xmax': 608, 'ymax': 359}, {'class': 'RBC', 'xmin': 566, 'ymin': 152, 'xmax': 640, 'ymax': 265}, {'class': 'RBC', 'x

100%|██████████| 60/60 [00:00<00:00, 940.05it/s]

{'bboxes': [{'class': 'RBC', 'xmin': 68, 'ymin': 154, 'xmax': 165, 'ymax': 249}, {'class': 'RBC', 'xmin': 1, 'ymin': 145, 'xmax': 66, 'ymax': 260}, {'class': 'RBC', 'xmin': 207, 'ymin': 160, 'xmax': 334, 'ymax': 270}, {'class': 'RBC', 'xmin': 435, 'ymin': 347, 'xmax': 540, 'ymax': 437}, {'class': 'RBC', 'xmin': 535, 'ymin': 356, 'xmax': 639, 'ymax': 464}, {'class': 'RBC', 'xmin': 446, 'ymin': 221, 'xmax': 550, 'ymax': 329}, {'class': 'RBC', 'xmin': 509, 'ymin': 94, 'xmax': 640, 'ymax': 207}, {'class': 'RBC', 'xmin': 456, 'ymin': 3, 'xmax': 560, 'ymax': 111}, {'class': 'RBC', 'xmin': 354, 'ymin': 144, 'xmax': 458, 'ymax': 252}, {'class': 'RBC', 'xmin': 169, 'ymin': 317, 'xmax': 277, 'ymax': 455}, {'class': 'RBC', 'xmin': 64, 'ymin': 302, 'xmax': 166, 'ymax': 406}, {'class': 'RBC', 'xmin': 74, 'ymin': 200, 'xmax': 176, 'ymax': 304}, {'class': 'RBC', 'xmin': 141, 'ymin': 18, 'xmax': 217, 'ymax': 109}, {'class': 'RBC', 'xmin': 316, 'ymin': 47, 'xmax': 409, 'ymax': 141}, {'class': 'RBC', 'x

In [ ]:
train_images = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Training/Images', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Training/Images')]
train_annotations = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Training/Annotations', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Training/Annotations') if x[-3:] == "txt"]

test_images = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Testing/Images', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Testing/Images')]
test_annotations = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Testing/Annotations', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Testing/Annotations') if x[-3:] == "txt"]

val_images = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Validation/Images', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Validation/Images')]
val_annotations = [os.path.join('/content/Complete-Blood-Cell-Count-Dataset/Validation/Annotations', x) for x in os.listdir('/content/Complete-Blood-Cell-Count-Dataset/Validation/Annotations') if x[-3:] == "txt"]


train_images.sort()
train_annotations.sort()

test_images.sort()
test_annotations.sort()

val_images.sort()
val_annotations.sort()


In [ ]:
import os 
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/train")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/val")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/train")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/val")
os.mkdir("/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/test")
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/train/')
move_files_to_folder(val_images, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/val/')
move_files_to_folder(test_images, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test/')
move_files_to_folder(train_annotations, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/train/')
move_files_to_folder(val_annotations, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/val/')
move_files_to_folder(test_annotations, '/content/Complete-Blood-Cell-Count-Dataset/new_dataset/labels/test/')



https://docs.ultralytics.com/modes/train/#arguments

In [ ]:
#cerete file contain pathes for dataset and classes 
with open('cbc_data.yaml', 'w') as f:
    f.write('train: ../Complete-Blood-Cell-Count-Dataset/new_dataset/images/train\n')
    f.write('val: ../Complete-Blood-Cell-Count-Dataset/new_dataset/images/val\n')
    f.write('test: ../Complete-Blood-Cell-Count-Dataset/new_dataset/images/test\n')
    f.write('nc: 3\n')
    f.write('names: ["RBC","WBC", "Platelets"]\n')


##Train phase

In [ ]:
!python /content/yolov5/train.py  --img 416 --batch 32 --epochs 2 --data /content/cbc_data.yaml --weights yolov5s.pt


train: weights=yolov5s.pt, cfg=, data=/content/cbc_data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=2, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-159-g1d65a5a Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.

##Export phase

In [ ]:
!python  /content/yolov5/export.py  --weights /content/yolov5/runs/train/exp/weights/best.pt --include tflite --img 416


export: data=yolov5/data/coco128.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v7.0-159-g1d65a5a Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/yolov5/runs/train/exp/weights/best.pt with output shape (1, 10647, 8) (13.6 MB)

TensorFlow SavedModel: starting export with tensorflow 2.12.0...

                 from  n    params  module                                  arguments                     
2023-05-01 12:03:00.941804: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is 

##Predict phase

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp/weights/best.torchscript  --img 640 --conf 0.25 --source /content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test/BloodImage_00347.jpg

detect: weights=['/content/yolov5/runs/train/exp/weights/best.torchscript'], source=/content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test/BloodImage_00347.jpg, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-158-g8211a03 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Loading /content/yolov5/runs/train/exp/weights/best.torchscript for TorchScript inference...
image 1/1 /content/Complete-Blood-Cell-Count-Dataset/new_dataset/images/test/BloodImage_00347.jpg: 640x640 62 RBCs, 11.5ms
Speed: 0.8ms pre-process, 11.5ms inference, 1.7m